Prototype of model explanation via LIME with help of extractive summary
---

In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

import time

tfds.disable_progress_bar()

In [2]:
now = time.strftime("%Y-%m-%d_%H:%M")

In [4]:
import logging
import sys

logging.basicConfig(
    level=logging.DEBUG, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename=f'../../data/logs/v1-{now}.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

l = logging.getLogger('prototype')
l.critical("Logging prototype v1 with TF model")

[{<ipython-input-4-e121ba337418>:14} CRITICAL - Logging prototype v1 with TF model


---

In [5]:
def _load_model():
    """
    Define a function that loads a model to be explained and returns its instance
    """
    
    dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    train_dataset.element_spec
    BUFFER_SIZE = 10000
    BATCH_SIZE = 64
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    
    VOCAB_SIZE=1000
    encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
    encoder.adapt(train_dataset.map(lambda text, label: text))
    
    model = tf.keras.Sequential([
        encoder,
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=64,
            # Use masking to handle the variable sequence lengths
            mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    model.load_weights('../../raw-data/lstm-model-v1')
    
    return model    

In [6]:
model = _load_model()
l.info("Model loaded")

[{dataset_builder.py:840} INFO - No config specified, defaulting to first: imdb_reviews/plain_text
[{dataset_info.py:362} INFO - Load dataset info from /home/tomasmizera/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
[{dataset_builder.py:323} INFO - Reusing dataset imdb_reviews (/home/tomasmizera/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
[{dataset_builder.py:528} INFO - Constructing tf.data.Dataset for split None, from /home/tomasmizera/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
[{<ipython-input-6-619c90ce7f16>:2} INFO - Model loaded


In [110]:
model.predict(["hahahahahahahahahaha this is the worst funny film I have ever seen"])

array([[-1.4518355]], dtype=float32)

In [8]:
model.predict_proba(["hahahahahahahahahaha this is the most funny film I have ever seen"])

Instructions for updating:
Please use `model.predict()` instead.
[{deprecation.py:317} WARNING - From <ipython-input-8-a7e6831c0071>:1: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.
[{sequential.py:425} WARNING - Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


array([[1.201608]], dtype=float32)

---

Explanation
---

### 1. Preparation

In [158]:
from lime import lime_text

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers import _summarizer

import os

In [513]:
summarizer = TextRankSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words('english')

In [14]:
explanator = lime_text.LimeTextExplainer(class_names=['negative', 'positive'])

---

### 2. Execution

In [109]:
l.info("Starting an algorithm")

[{<ipython-input-109-9e84a1d64b6b>:1} INFO - Starting an algorithm


In [557]:
# helper function to join summary
def _get_data_from_summary(summary):
    return ' '.join(list(map(lambda sentence: str(sentence), summary)))

----

In [235]:
INPUT_DIR = "/home/tomasmizera/school/diploma/src/data/reviews"

LANGUAGE = "english"
SENTENCES_COUNT = 6
TOP_FEATURES_COUNT = 10 

In [371]:
from functools import reduce

In [19]:
# define a decorator to log execusion time
# inspired by https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d

def timeit(method):
    def timed(*args, **kw):
        timed.calls += 1
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        timed.time_taken += (te - ts) * 1000
        return result
    timed.calls = 0
    timed.time_taken = 0
    return timed

In [112]:
@timeit
def _predict_proba(_input):
    """
    Define a function that accepts array of instances and returns a probability for each class 
    _input - 1d array of instances
    Returns 2d array of [num of instances] x [num of classes]
    """
    # TODO: implement this!
    # Mocked to simulate what LIME does in the background https://github.com/marcotcr/lime/issues/35
#     l.debug("Called _predict_proba with input: " + str(_input))
    return np.array(5000 * [[0.4, 0.6]])

In [114]:
def _explain_instance(_file, _explanator):
    explanation = _explanator.explain_instance(_file, _predict_proba, num_features=NUM_TOP_FEATURES)
    l.info('_explain_instance took:  %2.2f ms' % \
                  (_predict_proba.time_taken))
    l.info(f'_explain_instance called {_predict_proba.calls} times')
    _predict_proba.calls = 0
    _predict_proba.time_taken = 0
    
    return explanation

In [222]:
expl = _explain_instance(test_input)

[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.02 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times


**Some examples of perturbed text**: \[ ..., ' great hm', ' great ', ' great hm', '  hm', '  ', '  hm', '  ', '  ', '  ', '  ', 'not great ', '  ', '  hm', '  ', '  ', 'not great ', '  ', ' great ', ' great ', '  hm', '  ', ' great ', 'not  hm', ' great hm', ' great ', ' great hm', '  hm', 'not  ', '  hm', 'not  ', ' great hm', 'not great ', ' great ', '  ', '  ', '  hm', '  ', '  ', '  ', 'not  hm', 'not  ', 'not great ', 'not  hm', 'not  hm', 'not great ', 'not  hm', '  ', ' great ', '  ', '  hm', 'not  hm', '  ', '  ', ' great ', '  ', '  ', ' great ', 'not  ', 'not  hm', ' great ', 'not  ', 'not  ', 'not  hm', 'not  ', '  ', 'not great ', '  hm', ' great hm', '  hm', '  ', '  ', 'not  hm', '  hm', 'not  ', '  ', ' great hm', 'not  ', ' great hm', 'not  hm', 'not  ', 'not  hm', '  ', ' great ', '  hm', ' great ', 'not  hm', 'not  ', 'not  ', ' great hm', 'not  hm', '  hm', '  hm', ' great ', '  ', 'not great ', '  hm', 'not great ', '  ', '  ', 'not  hm', 'not great ', '  ', '  ', ' great hm', 'not  hm', 'not  hm', ' great ', ' great hm', '  ', 'not  hm', ' great ', '  hm', ' great ', 'not great ', '  ', '  ', ' great hm', ' great hm', '  ', ' great ', 'not  hm', ' great ', 'not  ', ' great ', 'not great ', ' great ', 'not  ', 'not great ', '  ', '  ', '  ', 'not  ', ' great hm', ' great hm', '  ', '  ', 'not great ', '  ', '  ', 'not  ', ' great ', ' great ', 'not great ', '  ', '  ', '  ', '  ', '  ', '  ', '  hm', '  hm', 'not  ', 'not  hm', 'not  ', '  ', ' great ', '  hm', ' great hm', '  ', '  ', '  ', ' great hm', '  ', '  ', 'not  hm', '  ', ' great hm', ' great hm', ' great ', '  ', '  ', 'not  ', '  ', 'not  ', ' great hm', 'not great ', ' great hm', 'not  hm', 'not great ', 'not  ', 'not great ', '  ', 'not great ', '  hm', 'not  ', ' great ', '  ', '  ', ' great ', '  hm', 'not  ', 'not  ', 'not  ', '  ', ' great hm', ' great hm', ' great ', ' great ', 'not great ', ' great ', '  ', 'not great ', 'not great ', '  ', 'not  ', ' great ', ' great ', '  ', '  ', ' great ', 'not  ', ' great ', '  ', '  ', ' great ', 'not  hm', 'not  ', '  ', 'not great ', '  ', '  ', '  ', 'not great ', 'not great ', '  ', '  hm', '  hm', 'not  hm', 'not great ', ' great ', 'not  ', '  ', 'not  hm', 'not great ', 'not  ', 'not great ', 'not  hm', 'not  ', 'not  hm', ' great hm', ' great ', '  hm', '  ', '  hm', '  ', 'not  ', ' great ', '  ', '  hm', 'not  hm', 'not great ', '  ', '  ', ' great ', '  ', '  ', 'not  ', 'not  ', ' great ', '  ', 'not  hm', '  ', ' great hm', '  ', '  ', '  ', ' great ', ' great ', ' great ', ' great ', ' great ', '  hm', 'not  ', ' great ', 'not  hm', '  ', ' great hm', 'not great ', '  hm', '  hm', 'not  hm', 'not  ', 'not great ', '  ', '  ', '  ', 'not  ', '  ', ' great ', '  ', 'not great ', '  ', 'not  ', '  ', '  hm', ' great ', '  ', 'not  ', ' great hm', '  ', ' great ', 'not great ', '  ']

In [97]:
# im_words = expl.as_list() TODO: when predict_proba will work
# $store -r a
im_words = a

In [542]:
def input_from_files(path_to_files):
    """
    Loads all readable files in path_to_files directory
    Returns np.array with each files content as a separate element
    """
    
    def _read_text_file(filepath):
        with open(filepath, 'r') as f:
            return reduce(lambda a, b: a + b, f.readlines())
    
    files_it = os.scandir(path_to_files)
    files_contents = {}
    
    for file in files_it:
        if file.is_file(): 
            files_contents[file.name] = _read_text_file(file.path)
        
    return files_contents

In [543]:
npInput = input_from_files(INPUT_DIR)

In [540]:
def _summarize_doc_custom(_summarizer, _instance, _explanation):
    """
    Returns summary with altered weights based on explanation
    _summarizer - summy summarizer instance
    _instance - instance content string
    _explanation - LIME explanation
    """
    
    def _create_weight_graph(_summarizer, _instance_doc):
        return _summarizer.rate_sentences(_instance_doc)
    
    def _count_factor(_sentence, _explanation_words_weight) -> float: # returns boosting factor for sentence
        factor = 1.0
        exp_words = list(map(lambda x: x[0], _explanation_words_weight))
        for word in _sentence.words:
            if word in exp_words:
                factor += abs(_explanation_words_weight[exp_words.index(word)][1])        
        return factor
    
    parser = PlaintextParser.from_string(_instance, Tokenizer(LANGUAGE))
    graph = _create_weight_graph(_summarizer, parser.document)
    
    for sentence in graph.keys():
        factor = _count_factor(sentence, _explanation.as_list())
        graph[sentence] = graph[sentence] * factor # TODO: normalize the factor value
        
    resulting_summary = _summarizer._get_best_sentences(parser.document.sentences, SENTENCES_COUNT, graph)
    
    return resulting_summary

In [574]:
def _summary_to_string(_summary):
    if len(_summary) <= 0:
        return ""
    
    summary_str = str(_summary[0])
    i = 1
    
    while(i < len(_summary)):
        summary_str += ' ' + str(_summary[i])
        i += 1
        
    return summary_str

In [576]:
def create_explanation_summaries(_instance_map, _explanator, _summarizer):
    """
    Returns summaries for all input elements
    _instance_map - map containing instance name and its content
    _explanator - LIME explanator instance
    _summarizer - summy summarizer instance
    """
    
    summaries = {}
    
    for instance in _instance_map.keys():
        explanation = _explain_instance(_instance_map[instance])
        summary = _summarize_doc_custom(_summarizer, _instance_map[instance], explanation)
        summaries[instance] = (_summary_to_string(summary), explanation.as_list())

    return summaries

In [575]:
def create_simple_summaries(_instance_map, _summarizer):
    """
    Returns summaries for all input instances
    _instance_map - map containing instance name and its content
    _summarizer - summy summarizer instance
    """
    
    summaries = {}
    
    for instance in _instance_map.keys():
        _parser = PlaintextParser.from_string(_instance_map[instance], Tokenizer(LANGUAGE))
        summaries[instance] = _summary_to_string(_summarizer(_parser.document, SENTENCES_COUNT))
    
    return summaries

In [599]:
from IPython.core.display import display, HTML

highlight_start = f'<mark style="background-color:rgba(250, 100, 0, {0.00})">'
highlight_end = '</mark>'
text = ''.join([highlight_start, "Hellloo!", highlight_end])

display(HTML(text))

In [589]:
def display_highlighted_summary(summary_pair):
    pass

In [577]:
create_explanation_summaries(npInput, explanator, summarizer);

[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.03 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.06 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.04 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.05 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.05 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.03 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.05 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} INFO - Took:  1.05 ms
[{<ipython-input-114-d3ff395e95d2>:5} INFO - Called 1 times
[{<ipython-input-114-d3ff395e95d2>:3} IN

In [582]:
create_simple_summaries(npInput, summarizer);

TODO:
- [ ] find a good pytorch/tf LSTM text classification model ~ maybe check datasets in LIME paper
- [ ] create predict_proba based on the type of the framework
- [ ] predict on created summaries ~ automatically -> (possible: save summaries to files and then load and pass them just as normal instance) 
- [ ] refactor process to not store everything in RAM, rather put intermediate results to files
- [ ] highlighting of important words from any summary (maybe save both, str summary and Sentence type summary - from sumy)
- [ ] extract it to separate script
- [ ] add better logging (more logs in this version)
- [ ] build and test quantitative experiment pipeline
- [ ] run quantitative experiment on all instances
- [ ] pick several (~6) explanations for user-study